In [1]:
#env anaconda3 python 3.11.7 - leticia
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from datetime import date

In [2]:
# READ CSV FILE 
path= r'\\10.93.162.41\mrtlllab\projects_progress\Robustness_pn\Paper_devtemp\data\ecdysone_qPCR\2403_secondtrial'
csv_name='qPCR_ecdysone_all_genes_raw.csv'
raw_data= pd.read_csv(path + "/" + csv_name)
raw_data.head()

,Well,Omit,Sample,Target,Task,Dyes,Cq,Cq Conf,Amp Score,Amp Status,Annotated,Threshold,Baseline Start,Baseline End,Tm1,Tm2,Tm3,Tm4,Curve Quality,Result Quality Issues
0,A1,False,2530,GAPDH,Unknown,SYBR,20.947729156200793,0.990117,2.825785,Amp,False,98309.513955,3,11,85.026268,NaN,NaN,NaN,NaN,NaN
1,A2,False,2530,GAPDH,Unknown,SYBR,20.835758885163173,0.990180,2.835256,Amp,False,98309.513955,3,11,85.026268,NaN,NaN,NaN,NaN,NaN
2,A3,False,2530,GAPDH,Unknown,SYBR,20.965280845811115,0.990545,2.820061,Amp,False,98309.513955,3,12,85.026268,NaN,NaN,NaN,NaN,NaN
3,A5,False,2530,RPL,Unknown,SYBR,16.007318699234727,0.987557,2.823787,Amp,False,77860.662866,3,7,82.887802,NaN,NaN,NaN,NaN,NaN
4,A6,False,2530,RPL,Unknown,SYBR,16.198816859391513,0.985226,2.781106,Amp,False,77860.662866,3,7,83.000549,NaN,NaN,NaN,NaN,NaN


In [3]:
# MAKE A WORKING COPY OF DATA
data= raw_data.copy()
pd.set_option('display.max_rows', None)     # ACTIVATE IF YOU WANT TO SEE ALL THE ROWS OF THE DF

# KEEP ONLY THE IMPORTANT COLUMNS
data = data.drop(columns=["Well", "Omit", "Task", "Dyes", "Cq Conf",'Amp Score', 'Amp Status', 'Annotated', 'Threshold', 'Baseline Start', 'Baseline End', 'Tm1', 'Tm2', 'Tm3', 'Tm4', 'Curve Quality', 'Result Quality Issues'])
data

# DROP 'NO DNA' SAMPLES
#second trial 
#data = data.loc[(data['Sample'] != 'RPL + WATER') & (data['Sample'] != 'ACT+ WATER') & (data['Sample'] != 'H1-55H')]
#print(data['Sample'].unique())

#first trial
# DROP 'NO DNA' SAMPLES
data = data.loc[(data['Sample'] != 'NO DNA')]
print(data['Sample'].unique())

['2530' '2555' '2580' '1866' '18121' '18176']


In [4]:
#data_test = data.loc[(data['Sample'] == 'H1-24H')  | (data['Sample'] == 'H2-24H') ]
data_test = data

In [5]:
ref_table = pd.DataFrame()
reference = '2530'

#gets mean delta ct 
ref = data_test.loc[(data_test['Sample'] == reference)]
for tar in ref['Target'].unique():
   #mean delta delta CT should be in relation to the CT value of 2530 for that gene
   ref_tar = ref.loc[(ref['Target'] == tar)]
   ref_rpl = ref.loc[(ref['Target']=='RPL')]

   #calculates delta ct of gene in relation to RPL
   #ref RPL mean
   ref_tar['rpl_mean'] = np.mean(pd.to_numeric(ref_rpl['Cq']))
   ref_tar['delta_ct'] = pd.to_numeric(ref_tar['Cq']) - ref_tar['rpl_mean']
   ref_tar['mean_delta_ct'] = np.mean(ref_tar['delta_ct'])
   ref_table = pd.concat([ref_table, ref_tar])



C:\Users\lleandro\AppData\Local\Temp\ipykernel_10900\904475766.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ref_tar['rpl_mean'] = np.mean(pd.to_numeric(ref_rpl['Cq']))
C:\Users\lleandro\AppData\Local\Temp\ipykernel_10900\904475766.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ref_tar['delta_ct'] = pd.to_numeric(ref_tar['Cq']) - ref_tar['rpl_mean']
C:\Users\lleandro\AppData\Local\Temp\ipykernel_10900\904475766.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [29]:
genes_table = pd.DataFrame()
results_table = pd.DataFrame()

for i in data_test['Sample'].unique():
   #print(i)
   df = data_test.loc[(data_test['Sample'] == i)]
   rpl = df.loc[(df['Target']=='RPL')]
   df['rpl_mean'] = np.mean(pd.to_numeric(rpl['Cq']))

   for tar in df['Target'].unique():
      #print(tar)
      gene = df.loc[(df['Target'] == tar)]
      gene['delta_ct'] = pd.to_numeric(df['Cq']) - df['rpl_mean']
      gene['mean_delta_ct'] = np.mean(gene['delta_ct'])
      ref_mdct = ref_table.loc[(ref_table['Target'] == tar)]
      ref_mdct_u = ref_mdct['mean_delta_ct'].mean()
      gene['delta_delta_ct'] = gene['delta_ct'] - ref_mdct_u 
      gene['fold_change'] = np.power(2, -gene['delta_delta_ct'])
      gene['mean_foldchange'] = np.mean(gene['fold_change'])
      genes_table = pd.concat([genes_table, gene])

results_table = pd.concat([results_table, genes_table])

results_table.to_csv('results.csv')




C:\Users\lleandro\AppData\Local\Temp\ipykernel_10900\2515566509.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rpl_mean'] = np.mean(pd.to_numeric(rpl['Cq']))
C:\Users\lleandro\AppData\Local\Temp\ipykernel_10900\2515566509.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene['delta_ct'] = pd.to_numeric(df['Cq']) - df['rpl_mean']
C:\Users\lleandro\AppData\Local\Temp\ipykernel_10900\2515566509.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u